In [5]:
#import python packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import pickle

%matplotlib inline

In [6]:
#import EKG analysis packages
import challengeOld as challenge
import waveOld as wave

In [7]:
records = wave.getRecords('~', _not=True)

In [8]:
feat_list = []
for record in records[0]:
    signal = challenge.Signal(record, wave.load(record))
    feat_list.append(challenge.feature_extract(signal))


In [9]:
labels=[]
for label in records[1]:
    if label == 'N':
        labels.append(0)
    elif label == 'O':
        labels.append(1)
    elif label == 'A':
        labels.append(2)
    elif label == '~':
        labels.append(3)

In [10]:
len(feat_list[0])

58

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feat_list, labels, test_size=0.20, random_state=42)

In [41]:
from sklearn.preprocessing import scale
X_train = scale(X_train)
X_test = scale(X_test)

In [42]:
from sklearn.decomposition import PCA
pca = PCA(n_components=16)
pca.fit(X_train)

X_train_PCA = pca.transform(X_train)



In [43]:
print (len(X_train_PCA))
print (np.sum(pca.explained_variance_ratio_))

6595
0.907729090311


In [44]:
pca.fit(X_test)
X_test_PCA = pca.transform(X_test)

In [45]:
print (len(X_train_PCA))

6595


In [53]:
EPOCHS = 100
BATCH_SIZE = 128
learning_rate = 0.0001
num_examples = len(X_train)

In [54]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.int32)
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 4)

In [71]:
fw1 = tf.Variable(tf.truncated_normal([16, 40], mean=0, stddev=0.1))
fb1 = tf.Variable(tf.zeros([40]))
fw2 = tf.Variable(tf.truncated_normal([40, 40], mean=0, stddev=0.1))
fb2 = tf.Variable(tf.zeros([40]))
fw3 = tf.Variable(tf.truncated_normal([40, 4], mean=0, stddev=0.1))
fb3 = tf.Variable(tf.zeros([4]))


In [72]:
#fully connect layer 1, 58 => 80
fully_layer_1 = tf.add(tf.matmul(X, fw1), fb1)
fully_layer_1 = tf.nn.relu(fully_layer_1)
#fully_layer_1 = tf.nn.dropout(fully_layer_1, keep_prob)

#fully connect layer 2, 80 => 80
fully_layer_2 = tf.add(tf.matmul(fully_layer_1, fw2), fb2)
fully_layer_2 = tf.nn.relu(fully_layer_2)
#fully_layer_2 = tf.nn.dropout(fully_layer_2, keep_prob)

#output layer => 80 => 4
logits = tf.add(tf.matmul(fully_layer_2, fw3), fb3)


In [73]:
#train the model
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss_operation)

#accuracy and loss
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(one_hot_y,1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [74]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_X, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={X: batch_X, y: batch_y})
        total_accuracy += (accuracy * len(batch_X))
    return total_accuracy / num_examples

In [75]:
from sklearn.utils import shuffle
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X_train_PCA, y_train = shuffle(X_train_PCA, y_train)
    for i in range(EPOCHS):
        sess.run(training_op, feed_dict={X: X_train_PCA, y: y_train})
        
        validation_accuracy = evaluate(X_test_PCA, y_test)
        print ('EPOCHS {}..'.format(i+1))
        print ('validation_accuracy = {:.3f}'.format(validation_accuracy))
        print ()
        
    saver.save(sess, 'model')
    print ('model_saved')

EPOCHS 1..
validation_accuracy = 0.054

EPOCHS 2..
validation_accuracy = 0.058

EPOCHS 3..
validation_accuracy = 0.061

EPOCHS 4..
validation_accuracy = 0.065

EPOCHS 5..
validation_accuracy = 0.068

EPOCHS 6..
validation_accuracy = 0.072

EPOCHS 7..
validation_accuracy = 0.075

EPOCHS 8..
validation_accuracy = 0.082

EPOCHS 9..
validation_accuracy = 0.088

EPOCHS 10..
validation_accuracy = 0.091

EPOCHS 11..
validation_accuracy = 0.098

EPOCHS 12..
validation_accuracy = 0.102

EPOCHS 13..
validation_accuracy = 0.107

EPOCHS 14..
validation_accuracy = 0.110

EPOCHS 15..
validation_accuracy = 0.121

EPOCHS 16..
validation_accuracy = 0.126

EPOCHS 17..
validation_accuracy = 0.130

EPOCHS 18..
validation_accuracy = 0.139

EPOCHS 19..
validation_accuracy = 0.144

EPOCHS 20..
validation_accuracy = 0.151

EPOCHS 21..
validation_accuracy = 0.155

EPOCHS 22..
validation_accuracy = 0.163

EPOCHS 23..
validation_accuracy = 0.171

EPOCHS 24..
validation_accuracy = 0.176

EPOCHS 25..
validation_ac

In [77]:
with tf.Session() as sess:
    
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    predictions = tf.argmax(logits,1)
    print(sess.run(predictions, feed_dict={X: X_test_PCA}))

[0 0 0 ..., 1 0 0]
